In [ ]:
# Ollama 설치 스크립트를 다운로드하여 실행합니다.
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# 1. 필수 라이브러리를 설치합니다.
!pip install -q ollama gradio

import os
import time
import threading
import subprocess
import gradio as gr
import ollama

# 2. Ollama 서버를 시작합니다.
os.system("pkill ollama")
threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]), daemon=True).start()
print("Ollama 서버를 준비 중입니다...")
time.sleep(15) # 서버가 켜질 때까지 대기
print("-> 서버 준비 완료.")

# 3. 사용할 모델을 다운로드합니다.
!ollama pull deepseek-r1:1.5b

# 4. 챗봇 답변 생성 함수를 정의합니다.
client = ollama.Client()

# prompt(현재 입력), history(과거 대화 기록), model(모델명)을 인자로 받습니다.
def generate_response(prompt, history, model):
    try:
        # Ollama API가 요구하는 형식으로 대화 기록을 변환합니다.
        messages = []
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

        # 현재 사용자의 메시지를 대화 기록에 추가합니다.
        messages.append({"role": "user", "content": prompt})

        # 전체 대화 기록을 모델에게 보내 답변을 생성하게 합니다.
        response = client.chat(model=model, messages=messages)
        return response['message']['content']
    except Exception as e:
        return f"오류: {e}"

# 5. Gradio 인터페이스를 생성하고, 백그라운드 스레드에서 실행합니다.
chatbot_ui = gr.ChatInterface(
    fn=generate_response,
    title="🤖 DeepSeek ChatBot",
    additional_inputs=[gr.Dropdown(choices=["deepseek-r1:1.5b"], value="deepseek-r1:1.5b", label="모델")]
)
threading.Thread(target=lambda: chatbot_ui.launch(), daemon=True).start()
print("-> Gradio 챗봇 인터페이스가 백그라운드에서 실행되었습니다.")
time.sleep(5) # Gradio 서버가 켜질 때까지 잠시 대기

# 6. Cloudflare 터널을 실행하여 외부 접속 주소를 생성합니다.
print("Cloudflare 터널을 실행하여 외부 접속 주소를 생성합니다...")
!pkill cloudflared
!wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared
!./cloudflared tunnel --url http://127.0.0.1:7860

Ollama 서버를 준비 중입니다...
-> 서버 준비 완료.



/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


-> Gradio 챗봇 인터페이스가 백그라운드에서 실행되었습니다.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2af03776aae5abf015.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Cloudflare 터널을 실행하여 외부 접속 주소를 생성합니다...
2025-08-08T01:15:03Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-08-08T01:15:03Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-08-08T01:15:09Z INF +--------------------------------------------------------------------------------------------+
2025-08-08T01:15:09Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-08-08T01:15:09Z INF |  https://grow-e